In [1]:
import os
import numba

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

In [5]:
from imutils import paths

In [29]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from keras.layers import AveragePooling2D, BatchNormalization
from keras.layers import Activation, AvgPool2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.layers import Input
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

In [10]:
initial_LR=1e-4
epoch=15
batch_size=8

In [11]:
data_loc='/home/mitra/Class/PSIT/Project/Maskdetection/Data/'
types=['mask_data','non_mask_data']

In [12]:
data=[]
labels=[]
for cat in types:
    path=os.path.join(data_loc, cat)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path, target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)
        data.append(image)
        labels.append(cat)

/home/mitra/anaconda3/lib/python3.8/site-packages/PIL/Image.py:962: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [13]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

In [14]:
data=np.array(data, dtype='float64')
labels=np.array(labels)

In [15]:
X_train, X_test, Y_train, Y_test=train_test_split(
data, labels, test_size=0.2, stratify=labels,
    random_state=42)

In [16]:
img_wid, img_hei, channel=224,224,3

In [18]:
mnv2=MobileNetV2(weights='imagenet',include_top=False,
input_tensor=Input(shape=(img_wid, img_hei, channel)))

In [19]:
main_model=mnv2.output
main_model=AveragePooling2D(pool_size=(7,7))(main_model)
main_model=Flatten(name='flatten')(main_model)
main_model=Dense(128,activation='relu')(main_model)
main_model=Dropout(0.5)(main_model)
main_model=Dense(2,activation='sigmoid')(main_model)

In [25]:
model=Model(inputs=mnv2.input,outputs=main_model)
for layer in mnv2.layers:
        layer.trainable=False

In [26]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',metrics=['accuracy'])

In [27]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

 block_3_expand_relu (ReLU)     (None, 56, 56, 144)  0           ['block_3_expand_BN[0][0]']      
                                                                                                  
 block_3_pad (ZeroPadding2D)    (None, 57, 57, 144)  0           ['block_3_expand_relu[0][0]']    
                                                                                                  
 block_3_depthwise (DepthwiseCo  (None, 28, 28, 144)  1296       ['block_3_pad[0][0]']            
 nv2D)                                                                                            
                                                                                                  
 block_3_depthwise_BN (BatchNor  (None, 28, 28, 144)  576        ['block_3_depthwise[0][0]']      
 malization)                                                                                      
                                                                                                  
 block_3_d

 lization)                                                                                        
                                                                                                  
 block_7_expand (Conv2D)        (None, 14, 14, 384)  24576       ['block_6_project_BN[0][0]']     
                                                                                                  
 block_7_expand_BN (BatchNormal  (None, 14, 14, 384)  1536       ['block_7_expand[0][0]']         
 ization)                                                                                         
                                                                                                  
 block_7_expand_relu (ReLU)     (None, 14, 14, 384)  0           ['block_7_expand_BN[0][0]']      
                                                                                                  
 block_7_depthwise (DepthwiseCo  (None, 14, 14, 384)  3456       ['block_7_expand_relu[0][0]']    
 nv2D)    

                                                                                                  
 block_10_depthwise_relu (ReLU)  (None, 14, 14, 384)  0          ['block_10_depthwise_BN[0][0]']  
                                                                                                  
 block_10_project (Conv2D)      (None, 14, 14, 96)   36864       ['block_10_depthwise_relu[0][0]']
                                                                                                  
 block_10_project_BN (BatchNorm  (None, 14, 14, 96)  384         ['block_10_project[0][0]']       
 alization)                                                                                       
                                                                                                  
 block_11_expand (Conv2D)       (None, 14, 14, 576)  55296       ['block_10_project_BN[0][0]']    
                                                                                                  
 block_11_

 block_14_depthwise (DepthwiseC  (None, 7, 7, 960)   8640        ['block_14_expand_relu[0][0]']   
 onv2D)                                                                                           
                                                                                                  
 block_14_depthwise_BN (BatchNo  (None, 7, 7, 960)   3840        ['block_14_depthwise[0][0]']     
 rmalization)                                                                                     
                                                                                                  
 block_14_depthwise_relu (ReLU)  (None, 7, 7, 960)   0           ['block_14_depthwise_BN[0][0]']  
                                                                                                  
 block_14_project (Conv2D)      (None, 7, 7, 160)    153600      ['block_14_depthwise_relu[0][0]']
                                                                                                  
 block_14_

In [30]:
aug=ImageDataGenerator(rotation_range=20,
                      zoom_range=0.2,
                      width_shift_range=0.2,
                      height_shift_range=0.2,
                      shear_range=0.15,
                      horizontal_flip=True,
                      fill_mode='nearest')

In [31]:
history=model.fit(aug.flow(X_train,
    Y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train)//batch_size,
                  validation_data=(X_test,Y_test),
        validation_steps=len(X_test)//batch_size,
        epochs=epoch)

Epoch 1/15
393/393 [==============================] - 55s 135ms/step - loss: 0.1066 - accuracy: 0.9650 - val_loss: 0.0255 - val_accuracy: 0.9936
Epoch 2/15
393/393 [==============================] - 41s 104ms/step - loss: 0.0574 - accuracy: 0.9787
Epoch 3/15
393/393 [==============================] - 41s 105ms/step - loss: 0.0531 - accuracy: 0.9838
Epoch 4/15
393/393 [==============================] - 43s 109ms/step - loss: 0.0412 - accuracy: 0.9866
Epoch 5/15
393/393 [==============================] - 43s 109ms/step - loss: 0.0409 - accuracy: 0.9860
Epoch 6/15
393/393 [==============================] - 43s 109ms/step - loss: 0.0415 - accuracy: 0.9863
Epoch 7/15
393/393 [==============================] - 43s 110ms/step - loss: 0.0299 - accuracy: 0.9898
Epoch 8/15
393/393 [==============================] - 43s 110ms/step - loss: 0.0266 - accuracy: 0.9911
Epoch 9/15
393/393 [==============================] - 43s 109ms/step - loss: 0.0289 - accuracy: 0.9898
Epoch 10/15
393/393 [==========

In [ ]:
pred=model.predict(X_test,batch_size=batch_size)
pred=np.argmax(pred, axis=1)

In [ ]:
model.save("mask_model", save_format="h5")